In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pandas_datareader.data as web
from urllib.request import urlopen
from bs4 import BeautifulSoup
import quandl
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing, model_selection , svm


In [ ]:
now_time = datetime.now()
if now_time.month != 1:
    start_time = datetime(now_time.year, now_time.month - 1, now_time.day)
else:
    start_time = datetime(now_time.year -1, 12, now_time.day)
print(start_time)


In [ ]:
# this will give  Date High Low Open Close Volume AjClose
# print(web.DataReader('AMZN', 'yahoo', start_time, now_time))
# IEX
# this will give  Date High Low Open Close Volume 
# print(web.DataReader("BAC", "iex", start_time, now_time))
print("====================")
print(quandl.get("WIKI/AMZN"))

In [ ]:
def get_american_stock_data(stock_of_interest, start_time, now_time):
    """Get stock price of american stock."""
    stock_df = web.DataReader(stock_of_interest, 'yahoo', start_time, now_time)
    return stock_df

stocks_of_interest = ['AMZN','GOOG','AAPL','CLDR','HDP','ORCL','TSLA']
for stock in stocks_of_interest:
    #scrape the stock data to a df
    stock_df = get_american_stock_dat(stock,start_time, now_time )

In [ ]:
def international_last_month_prices(market, stock):
    """ return a historical stock dataframe for any market and any stock """
    google_historical_price_site= 'https://www.google.ca/finance/historical?q='
    historical_price_page = google_historical_price_site+market+'%3A'+stock
    stock_dat = urlopen('https://ca.finance.yahoo.com/quote/APHA.TO/history?period1=1536361200&period2=1544227200&interval=1d&filter=history&frequency=1d')
    #then parse the table with BS 
    historical_page = BeautifulSoup(stock_dat,'lxml')
    table_dat = historical_page.find('table',{'class':'W(100%) M(0)'})
    rows = table_dat.findAll('td',{'class':'Py(10px) Ta(start) Pend(10px)'})
    dates = [x.get_text().rstrip() for x in rows]
    #get datetime formatted dates
    datetime_dates = [datetime.strptime(x, '%b %d, %Y') for x in dates]
    prices = []
    #iterate and grab column data
    for num, row in enumerate(rows):
        row_dat = [datetime_dates[num]] #first column is the dates
        for i in row.next_siblings: 
            row_dat.append(i.get_text().rstrip()) 
        prices.append(row_dat) #add the row to the list of rows
    outdat = pd.DataFrame(prices,columns = ['Date','Open','High','Low','Close','Adj Close','Volume'])
    print(outdat)
    #cleanup, set index and make volume integers
    outdat["Volume"] = outdat["Volume"].apply(lambda x: int(x.replace(',','').replace('-','0')) if x  else None)
    #change the other columns to floating point values
    
    for col in ['Open','High','Low','Close']:
        outdat[col] = outdat[col].apply(lambda x: float(x.replace('-','0')))
    outdat = outdat.set_index('Date')
    outdat = outdat.sort_index() #sort the index so it is oldest to newest
    return outdat

international_last_month_prices('FRA', 'LHA')

In [56]:
df = quandl.get("WIKI/AMZN")

df = df[["Adj. Close"]]

forecast_out = int(30) # predicting 30 days into future
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out) #  label column with data shifted 30 units up
print(df.tail())
X = np.array(df.drop(['Prediction'], 1))
X = preprocessing.scale(X)
X_forecast = X[-forecast_out:] # set X_forecast equal to last 30
X = X[:-forecast_out] # remove last 30 from X
print(X)

y = np.array(df['Prediction'])
y = y[:-forecast_out]
print(y)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.2)
# Training
clf = LinearRegression()
clf.fit(X_train,y_train)
# Testing
confidence = clf.score(X_test, y_test)
print("confidence: ", confidence)
forecast_prediction = clf.predict(X_forecast)
print(forecast_prediction)

            Adj. Close  Prediction
Date                              
2018-03-21     1581.86         NaN
2018-03-22     1544.10         NaN
2018-03-23     1495.56         NaN
2018-03-26     1555.86         NaN
2018-03-27     1497.05         NaN
[[-0.70904095]
 [-0.70911489]
 [-0.7093722 ]
 ...
 [ 4.07790031]
 [ 4.03715557]
 [ 4.20481944]]
[1.54166667e+00 1.51583333e+00 1.58833333e+00 ... 1.49556000e+03
 1.55586000e+03 1.49705000e+03]
confidence:  0.9885339272669885
[1500.10560829 1538.94410471 1550.32780194 1536.43565229 1557.33233646
 1572.81884147 1574.77458403 1590.97323443 1614.30396778 1603.70681919
 1604.20638387 1584.01121605 1591.23896032 1616.0683877  1630.98092478
 1638.80389505 1646.09541354 1674.82569702 1695.55231662 1684.70007118
 1687.69745924 1678.47145626 1667.16216228 1638.7294918  1682.92502222
 1677.98252062 1637.84728184 1586.25394258 1650.34702782 1587.8376689 ]


In [69]:
# Define my period
d1 = datetime(2018,8,1)
d2 = datetime(2018,12,1)

# Get the data
df = web.DataReader("GOOG", 'yahoo', d1, d2)
# Calculate some indicators
df['20d_ma'] = df['Adj Close'].rolling(20).mean()
df['50d_ma'] = df['Adj Close'].rolling(50).mean()
print (df[-5:])
# Create the model
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split

X = df[list(df.columns)[6:]] # Adj Close and indicators...
y = df['Adj Close']

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = LinearRegression()
model.fit(X_train,y_train)

                   High          Low         Open        Close   Volume  \
Date                                                                      
2018-11-26  1049.310059  1033.910034  1038.349976  1048.619995  1850400   
2018-11-27  1057.579956  1038.489990  1041.000000  1044.410034  1803200   
2018-11-28  1086.839966  1035.760010  1048.760010  1086.229980  2475400   
2018-11-29  1094.244995  1076.000000  1076.079956  1088.300049  1468900   
2018-11-30  1095.569946  1077.880005  1089.069946  1094.430054  2580200   

              Adj Close       20d_ma       50d_ma  
Date                                               
2018-11-26  1048.619995  1049.955008  1103.712191  
2018-11-27  1044.410034  1051.171509  1101.479391  
2018-11-28  1086.229980  1053.672510  1099.979591  
2018-11-29  1088.300049  1054.249011  1098.323793  
2018-11-30  1094.430054  1055.470514  1096.474994  


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [59]:
pd.__version__

'0.23.0'

AttributeError: module 'pandas' has no attribute 'rolling'